In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from tqdm.notebook import tqdm
tqdm.pandas()

In [62]:
data_dir = './kaggle/input'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [63]:
df_train

,Id,GuideSeq,Fraction_Insertions,Avg_Deletion_Length,Indel_Diversity,Fraction_Frameshifts
0,0,CTGCAGGGCTAGTTTCCTATAGG,0.069572,4.301844,3.536538,0.807375
1,1,GAGATGCGGACCACCCAGCTGGG,0.287647,10.814444,3.871165,0.665696
2,2,GCAAACGGAAGTGCAATTGTCGG,0.137004,9.888889,3.931298,0.684823
3,3,GTCATCGCTGAGTTGAGGAAGGG,0.093889,4.527812,3.523067,0.753003
4,4,ATATGATTATCCCTGCACAAGGG,0.526525,6.415644,2.828101,0.887214
...,...,...,...,...,...,...
1060,1060,ATGCCCGACCAAAGACAACCAGG,0.093236,3.641787,3.170302,0.684801
1061,1061,CACGCTGTCATCCACCAGGTAGG,0.069054,7.661831,4.284711,0.639433
1062,1062,GGGCTCCAGAGTCTGATACAGGG,0.421439,7.364856,3.175859,0.832975
1063,1063,AAAGACTTCGGTCCTCTAGTAGG,0.206277,7.907886,3.782588,0.528375


In [64]:
cat = pd.CategoricalDtype(categories=['A', 'T', 'C', 'G'])

letters = ['A', 'T', 'G', 'C']
max_length = 1


def add_cols(data: pd.DataFrame):
    seq_len = len(data['GuideSeq'].iloc[0])
    
    acc = data
    for l in tqdm(range(1, max_length+1)):
        for pos in tqdm(range(0, seq_len - l + 1)):
            indices = ':'.join(str(i) for i in range(pos, l+pos))
            col_name = f'pos_{l}_{indices}'
            col_data = data["GuideSeq"].str.slice(start=pos, stop=pos+l)
            acc = pd.concat([acc, col_data.rename(col_name).astype('category')], axis=1)
            # for combination in combinations:
            #     col_name_2 = col_name + "_is_" + combination
            #     col_data_2 = col_data == combination
            #     acc = pd.concat([acc, col_data_2.rename(col_name_2)], axis=1)
    return acc

# x = make_letters_cols(df_train)

# def add_cols(df: pd.DataFrame):
#     split_columns = df['GuideSeq'].apply(lambda x: pd.Series(list(x)))
#     split_columns.columns = [f'pos_{i+1}' for i in range(split_columns.shape[1])]
#     exp = pd.concat([df, split_columns], axis=1)
#     for col in exp.columns:
#         if col.startswith('pos_'):
#             exp[col] = exp[col].astype(cat)
#     return exp

data_expanded = add_cols(df_train)
test_data_expanded = add_cols(df_test)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

In [98]:
data_expanded.filter(regex='^pos_1')

,pos_1_0,pos_1_1,pos_1_2,pos_1_3,pos_1_4,pos_1_5,pos_1_6,pos_1_7,pos_1_8,pos_1_9,...,pos_1_13,pos_1_14,pos_1_15,pos_1_16,pos_1_17,pos_1_18,pos_1_19,pos_1_20,pos_1_21,pos_1_22
0,C,T,G,C,A,G,G,G,C,T,...,T,T,C,C,T,A,T,A,G,G
1,G,A,G,A,T,G,C,G,G,A,...,C,C,C,A,G,C,T,G,G,G
2,G,C,A,A,A,C,G,G,A,A,...,C,A,A,T,T,G,T,C,G,G
3,G,T,C,A,T,C,G,C,T,G,...,T,G,A,G,G,A,A,G,G,G
4,A,T,A,T,G,A,T,T,A,T,...,T,G,C,A,C,A,A,G,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,A,T,G,C,C,C,G,A,C,C,...,G,A,C,A,A,C,C,A,G,G
1061,C,A,C,G,C,T,G,T,C,A,...,A,C,C,A,G,G,T,A,G,G
1062,G,G,G,C,T,C,C,A,G,A,...,T,G,A,T,A,C,A,G,G,G
1063,A,A,A,G,A,C,T,T,C,G,...,C,T,C,T,A,G,T,A,G,G


In [106]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint

epsilon = 0.01
def get_model(X, y, std):
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test, std_train, std_test = train_test_split(X, y, std, test_size=0.2, random_state=42)

    param_distributions = {
        'n_estimators': randint(50, 2000),
        'max_depth': randint(3, 12),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0, 1),
        'colsample_bytree': uniform(0, 1),
        'reg_alpha': uniform(0, 1.0),
        'reg_lambda': uniform(1.0, 3.0)
    }
    sample_weights = 1 / (std_train + epsilon)

    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True)
    kf = KFold(n_splits=6, shuffle=True, random_state=42)

    random_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_distributions,
        n_iter=100,
        scoring='neg_mean_squared_error',
        cv=kf,
        random_state=42,
        n_jobs=-1,
        error_score="raise"
    )

    random_search.fit(X_train, y_train, sample_weight=sample_weights)

    best_model = random_search.best_estimator_
    # best_model = xgb_model
    best_model.fit(X_train, y_train, sample_weight=sample_weights)

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Test Mean Squared Error: {mse}")
    return best_model

features = ['Fraction_Insertions', 'Avg_Deletion_Length', 'Fraction_Frameshifts', 'Indel_Diversity']

models = {}

# models['Indel_Diversity'] = get_model(data_expanded.filter(regex='^pos_1'), data_expanded['Indel_Diversity'])

for feature in features:
    models[feature] = get_model(data_expanded.filter(regex='^pos_1'), data_expanded[feature], data_expanded['Indel_Diversity'])


In [17]:
features = ['Fraction_Insertions', 'Avg_Deletion_Length', 'Indel_Diversity', 'Fraction_Frameshifts']
X_train = data_expanded.filter(regex='^pos')
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                         'max_depth': 5, 'alpha': 10}
xgb_cv = xgb.cv(dtrain=data_expanded, nfold=5, params=params,
                        num_boost_round=50, early_stopping_rounds=10, metrics=features, as_pandas=True, seed=123)
# for feature in features:
#     y_train = data_expanded[feature]

#     params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
#                         'max_depth': 5, 'alpha': 10}



AttributeError: 'DataFrame' object has no attribute 'get_uint_info'

In [13]:
models['Fraction_Insertions'].get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': np.float64(0.7901755405312056),
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': True,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': np.float64(0.07059599747810114),
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 4,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 88,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': np.float64(0.6510770255019445),
 'reg_lambda': np.float64(3.7448790266313425),
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': np.float64(0.8500385777897993),
 'tree_m

In [93]:

t = test_data_expanded.filter(regex='^pos_|Id').copy()

t.insert(0, 'Indel_Diversity', models['Indel_Diversity'].predict(t.filter(regex='^pos_|Indel_Diversity')))
t['Fraction_Insertions'] = models['Fraction_Insertions'].predict(t.filter(regex='^pos_|Indel_Diversity'))
t['Avg_Deletion_Length'] = models['Avg_Deletion_Length'].predict(t.filter(regex='^pos_|Indel_Diversity'))
t['Fraction_Frameshifts'] = models['Fraction_Frameshifts'].predict(t.filter(regex='^pos_|Indel_Diversity'))
t[['Id', 'Fraction_Insertions', 'Avg_Deletion_Length', 'Indel_Diversity', 'Fraction_Frameshifts']].to_csv('./kaggle/output/out.csv', index=False)
t



,Indel_Diversity,Id,pos_1_0,pos_1_1,pos_1_2,pos_1_3,pos_1_4,pos_1_5,pos_1_6,pos_1_7,...,pos_1_16,pos_1_17,pos_1_18,pos_1_19,pos_1_20,pos_1_21,pos_1_22,Fraction_Insertions,Avg_Deletion_Length,Fraction_Frameshifts
0,3.973236,0,T,G,T,G,C,A,A,T,...,C,T,A,A,G,G,G,0.166636,6.099944,0.717813
1,3.146923,1,T,G,T,C,T,G,G,C,...,T,A,C,A,G,G,G,0.393697,5.898020,0.781891
2,3.625931,2,A,C,T,G,A,G,A,G,...,A,A,A,G,T,G,G,0.226644,6.118078,0.718868
3,3.681275,3,G,T,T,C,T,G,C,A,...,T,T,C,A,C,G,G,0.297023,8.540920,0.745794
4,4.099708,4,A,C,T,G,G,A,T,G,...,T,G,G,T,G,G,G,0.164510,7.676237,0.667048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,4.097219,299,T,A,T,G,A,A,G,C,...,G,A,G,T,C,G,G,0.128512,6.605610,0.667480
300,3.558367,300,A,T,G,T,T,G,G,A,...,C,G,G,T,G,G,G,0.144433,5.688961,0.713817
301,3.750374,301,G,A,C,C,A,G,G,G,...,T,T,A,A,A,G,G,0.228780,6.441158,0.708041
302,3.999604,302,A,G,T,A,G,A,C,T,...,A,G,G,A,G,G,G,0.217984,7.839652,0.697473
